# SimParam

> The SimParam class is the cornerstone of a simulation, acting as a global container for parameters that are not specific to any single Population but govern the rules of the entire simulation. Just as the Population class holds the state of individuals, SimParam holds the "genetic laws" of the simulated world, such as trait architecture, genetic maps, and SNP chip definitions.



In [ ]:
#| default_exp sp

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass, field
from typing import List, Optional, TYPE_CHECKING
from typing import Optional, TYPE_CHECKING

# No need for cached_property anymore for these core attributes
# from functools import cached_property

import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass, field
from typing import List, Optional, TYPE_CHECKING

if TYPE_CHECKING:
    from chewc.trait import TraitCollection, LociMap
    from chewc.population import Population

@flax_dataclass(frozen=True)
class SimParam:
    """A container for all global simulation parameters."""
    # --- JAX-Compatible Attributes (leaves of the Pytree) ---
    gen_map: jnp.ndarray
    ploidy: int
    traits: Optional['TraitCollection'] = None
    recomb_params: tuple = (2.6, 0.0, 0.0)
    pedigree: Optional[jnp.ndarray] = None
    var_e: Optional[jnp.ndarray] = None
    
    # --- Static, Non-Traceable Attributes ---
    # These are ignored by JAX transformations like jit and vmap.
    sexes: str = field(default='no', pytree_node=False)
    snp_chips: List['LociMap'] = field(default_factory=list, pytree_node=False)
    n_threads: int = field(default=1, pytree_node=False)
    track_pedigree: bool = field(default=False, pytree_node=False)
    track_recomb: bool = field(default=False, pytree_node=False)
    last_id: int = field(default=0, pytree_node=False)

    @classmethod
    def from_founder_pop(
        cls,
        founder_pop: 'Population',
        gen_map: jnp.ndarray,
        **kwargs
    ) -> 'SimParam':
        # Using direct shape access is more robust
        ploidy = founder_pop.geno.shape[2]
        n_ind = founder_pop.geno.shape[0]
        return cls(
            gen_map=gen_map,
            ploidy=ploidy,
            last_id=n_ind,
            **kwargs
        )


    # --- Derived Properties ---
    @property
    def centromere(self) -> jnp.ndarray:
        """Sets a default centromere position for each chromosome."""
        return jnp.zeros(self.n_chr)

    @property
    def n_chr(self) -> int:
        """Returns the number of chromosomes from the genetic map shape."""
        return self.gen_map.shape[0]

    @property
    def n_loci_per_chr(self) -> jnp.ndarray:
        """Returns an array with the number of loci for each chromosome."""
        return jnp.full((self.n_chr,), self.gen_map.shape[1])

    @property
    def n_traits(self) -> int:
        """Returns the number of defined traits."""
        if self.traits is None:
            return 0
        return self.traits.n_traits

    def __repr__(self) -> str:
        return (f"SimParam(nChr={self.n_chr}, nTraits={self.n_traits}, "
                f"ploidy={self.ploidy}, sexes='{self.sexes}')")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()